In [2]:
!pip install -U kaleido
!pip install plotly


     |████████████████████████████████| 79.9 MB 120 kB/s 


In [23]:
import numpy as np
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.io as pio

In [3]:
rng = np.random.RandomState(3)
figure="some_figure.pdf"
fig=px.scatter(x=[0, 1, 2, 3, 4], y=[0, 1, 4, 9, 16])
fig.write_image(figure, format="pdf")

# Question préliminaires

In [4]:
def sample_A_B(N, class_proba, C, q):
    """Sample the matrix."""
    qq_star = q.reshape(1, -1) * q.reshape(-1, 1)
    A = np.zeros_like(qq_star)
    nb_points_per_class = rng.multinomial(N, class_proba)
    
    labels = np.zeros((N, K),dtype=int)
    
    cum_ind = np.cumsum(nb_points_per_class)
    prev_ind = 0
    for i in range(0, len(cum_ind)):
        ind = cum_ind[i]
        labels[prev_ind:ind, i] = 1
        prev_ind = ind
    
    proba_matrix = np.diag(q) @ labels @ C @ labels.T @ np.diag(q)

    for i in range(A.shape[0]):
        for j in range(i, A.shape[1]):
            A[i, j] = rng.binomial(1, proba_matrix[i, j])
    A = A + A.T - np.identity(N) * A
    B = A - qq_star
    B_scaled = B / np.sqrt(N)
    return A, B_scaled, labels.T


In [5]:
def save_fig(fig_func):
    """Utility function to save the figure."""
    def wrapper(save_path=None, *args, **kwargs):
        fig = fig_func(*args, **kwargs)
        if save_path is not None:
            pio.write_image(fig, save_path)
    return wrapper

In [17]:
def generate_A_B(M, q, proba=None):
    """Generate the matrices with given class probabilities."""
    N = q.shape[0]
    K = M.shape[0]
    if proba is None:
        proba = rng.randint(1, 5, K)
        proba = proba / proba.sum()
    C = 1. + M / np.sqrt(N)
    A, B_scaled, class_vectors = sample_A_B(N, proba, C, q)
    return B_scaled, class_vectors

def plot_eigval_eigvec(
    M,
    q,
    q_value="",
    n_plot_vecs=5,
    proba=None,
    main_plot_width=0.5,
    nbins=200,
    colors=px.colors.qualitative.Plotly,
    return_eig=False,
    show=False,
    title="Distribution des valeurs propres avec"
):
    """Plot the eigenvalues distribution with the 5 first eigenvectors."""
    N, K = q.shape[0], M.shape[0]

    B_scaled, class_vectors = generate_A_B(M, q, proba)
    eigval, eigvec = np.linalg.eigh(B_scaled)
    column_widths = [main_plot_width] + [
        (1 - main_plot_width) / n_plot_vecs for _ in range(n_plot_vecs)
    ]
    subplot_titles = [""] + [rf"$\lambda_{{{i+1}}}$" for i in range(n_plot_vecs)]
    fig = make_subplots(
        1, n_plot_vecs + 1, column_widths=column_widths, subplot_titles=subplot_titles
    )
    data_eigval = px.histogram(
        x=eigval, nbins=nbins, color_discrete_sequence=["firebrick"],
    ).data[0]

    fig.add_trace(data_eigval, row=1, col=1)

    # Sort the eigenvalues by decreasing module
    sorted_eig_ind = np.argsort(np.abs(eigval))[::-1]
    eigval_sorted = eigval[sorted_eig_ind]

    N = eigval.shape[0]

    eigvec_sorted = eigvec[:, sorted_eig_ind]
    n_colors = len(colors)
    for i in range(0, n_plot_vecs):
        data_eigvecs = go.Scatter(
            x=eigvec_sorted[:, i],
            y=np.arange(N),
            line=dict(color=colors[(i + 5) % n_colors]),
        )
        fig.add_trace(data_eigvecs, row=1, col=i + 2)

    fig.data[0].marker.line.width = 1
    fig.data[0].marker.line.color = "white" 
    fig.update_layout(
        margin=dict(t=50, b=5, r=5, l=5),
        title_text=title,
        width=1100,
        height=500,
        showlegend=False,
        font_size=14,
        title_font_size=20
    )
    if show:
        fig.show()
    if return_eig:
        return fig, eigval, eigvec, class_vectors
    return fig


def project_and_3D_plot(B_scaled, class_vectors, q_title="", show=True):
        eigval, eigvec = np.linalg.eigh(B_scaled)
        B_projected = eigvec[:, -3:].T @ B_scaled

        colors_vectors = np.arange(K)[:, None] * class_vectors
        colors_vectors = colors_vectors.sum(axis=0)

        fig = px.scatter_3d(x=B_projected[0], y=B_projected[1], z=B_projected[2], color=colors_vectors.astype(str))
        fig.update_layout(legend_title_text="class")
        fig.update_layout(
                margin=dict(t=50, b=5, r=5, l=5),
                title_text=rf"Répartition selon les 3 directions principales",
                width=800,
                height=500,
                font_family="Serif",
                font_size=14,
                title_font_size=20
            )
        fig.update_traces(marker_size=4)
        fig.update_yaxes(title="")
        fig.update_xaxes(title="")
        if show:
            fig.show()
        return fig

@save_fig
def plot_eigendirections(M, q, proba, q_title):
    B_scaled, class_vectors = generate_A_B(M, q, proba)

    fig = project_and_3D_plot(B_scaled, class_vectors, q_title)
    return fig


In [7]:
@save_fig
def plot_several_M(
    list_M, M_titles, q, proba, q_value, n_plot_vecs=5, main_plot_width=0.5, nbins=200
):
    N = q.shape[0]
    K = list_M[0].shape[0]

    list_fig = [plot_eigval_eigvec(M, q, proba=proba) for M in list_M]
    n_fig = len(list_fig)
    column_widths = [main_plot_width] + [
        (1 - main_plot_width) / n_plot_vecs for _ in range(n_plot_vecs)
    ]
    subplot_titles = []
    for k in range(n_fig):
        sub_title = [M_titles[k]] + [
            rf"$\lambda_{{{i+1}}}$" for i in range(n_plot_vecs)
        ]
        subplot_titles.extend(sub_title)
    fig = make_subplots(
        n_fig,
        n_plot_vecs + 1,
        column_widths=column_widths,
        subplot_titles=subplot_titles,
        vertical_spacing=0.05
    )
    n_figures_per_row = n_plot_vecs + 1

    for k in range(n_fig):
        data = list_fig[k].data
        fig.add_traces(
            data,
            rows=[k + 1 for i in range(n_figures_per_row)],
            cols=[i + 1 for i in range(n_figures_per_row)],
        )

    fig.update_layout(
        margin=dict(t=50, b=5, r=5, l=5),
        width=1100,
        height=350 * n_fig,
        showlegend=False,
        font_family="Serif",
        font_size=14,
    )
    fig.update_annotations(font_size=20)
    fig.show()
    return fig

## $q_i$ uniforme dans $[0.1, 0.5]$.

In [ ]:
N = 1500
K = 3
M_diag_pos = 50 * np.diag(rng.rand(K))
M_diag_neg = -20 * np.diag(rng.rand(K))
M_full_50 = 30 * rng.rand(K, K)
M_full_10 = 5 * rng.rand(K, K)
list_M = [M_diag_pos, M_diag_neg, M_full_50, M_full_10]
q = .4 * rng.rand(N) + .1
proba = rng.randint(1, 5, K)
proba = proba / proba.sum()

q_value = r"q_i \text{uniforme dans}\,[0.1, 0.5]"
M_titles = ["M diagonale positive ordre de grandeur 50", "M diagonale negative ordre de grandeur 20", "M complète ordre de grandeur 30", "M complète ordre de grandeur 5"]
fig_uniform = plot_several_M(save_path="q-uniform.pdf", list_M=list_M, M_titles=M_titles, q=q, proba=proba, q_value=q_value)


In [ ]:
plot_eigendirections("3d-uniform.pdf", list_M[0], q, proba, q_value) 

## $q_i = q_0$, valeur constante.

In [ ]:
q_0 =  0.6
q = q_0 * np.ones(N)
q_value = fr"\forall i, q_i = {{{q_0}}}"
plot_several_M(save_path="q-constant.pdf", list_M=list_M, M_titles=M_titles, q=q, proba=proba, q_value=q_value)

Let's see the repartition on the first eigendirections. We take $M$ corresponding to the first figure and hence expect 2 main eigendirections.

In [ ]:
plot_eigendirections("3d-constant.pdf", list_M[0], q, proba, q_value)

Classes très bien séparées.

## $q_i \in [0.4, 0.1]$, deux valeurs.

In [ ]:
q_choices = [0.4, 0.1]
q = rng.choice(q_choices, N)
q_value = r"\forall i, q_i \in \{0.4, 0.1\}"

In [ ]:
plot_several_M(save_path="2values.pdf", list_M=list_M, M_titles=M_titles, q=q, proba=proba, q_value=q_value)

Les formes des vecteurs propres semblent un peu moins claires que précédemment, et la forme est très éloignée de celle d'une matrice de Wigner.

In [ ]:
plot_eigendirections("3d-2values.pdf", list_M[0], q, proba, q_value) 

On semble observer pour chaque classe deux groupes, correspondant probablement aux deux valeurs de $q_i$.

# Homogène

In [ ]:
N = 3000
K = 3
rng = np.random.RandomState(0)
proba = rng.randint(1, 5, K)
proba = proba / proba.sum()
q0 =  0.4
q = q0 * np.ones(N)
M = 50 * np.diag(rng.rand(K))
sigma = np.sqrt(q0**2 * (1 - q0**2))



In [ ]:
fig, eigval, eigvec, class_vectors =  plot_eigval_eigvec(M, q, proba=proba, q_value=fr"\forall i, q_i = {{{q0}}}", return_eig=True, title="Distribution des valeurs propres avec leurs valeurs asymptotiques.")

In [ ]:
# Number of expected outliers eigenvalues
gamma = proba * q0 ** 2 * np.diag(M)
mask_outliers = gamma > sigma 
n_outliers = mask_outliers.sum()
print(f"Number of expected outliers eigenvalues: {n_outliers}")

Number of expected outliers eigenvalues: 3


In [ ]:
eigval_outliers = eigval[-n_outliers:]
eigvec_outliers = eigvec[:, -n_outliers:]
eigval_outliers

array([1.1526283 , 2.02625901, 3.90601154])

In [ ]:
predicted_lambda = gamma + (sigma**2) / gamma
ind_classes = np.argsort(predicted_lambda)
sorted_predicted_lambda = predicted_lambda[ind_classes]
sorted_class_vectors = class_vectors[ind_classes]
sorted_proba = proba[ind_classes]
sorted_m = np.diag(M)[ind_classes]
sorted_gamma = sorted_proba * q0 ** 2 * sorted_m

print("Theoretical asymptotical eigenvalues:", sorted_predicted_lambda[-n_outliers:])

Theoretical asymptotical eigenvalues: [1.10416778 2.00597626 3.95630502]


In [ ]:
for pred_eigval in sorted_predicted_lambda[-n_outliers:]:
    average_number = 2 * N/200  
    trace = go.Scatter(x=[pred_eigval, pred_eigval], y=[0, average_number], line=dict(color="lightslategrey"), marker=dict(size=[0, 10], opacity=1))
    fig.add_trace(trace)

fig.update_layout(font_family="Serif", font_size=14)

fig.show()
pio.write_image(fig, "homogene-eigval.pdf")

In [ ]:
@save_fig
def plot_eigvec_labels(
    eigvecs,
    labels,
    theoretical_alignments,
    real_alignments,
    colors=px.colors.qualitative.Plotly,
):
    n_rows = eigvecs.shape[1]
    subplot_titles = []
    for i in range(n_rows):
        subplot_titles.append(f"Class {i}")
        subplot_titles.append("Alignments comparison" if i == 0 else "")
    fig = make_subplots(rows=n_rows, cols=2, subplot_titles=subplot_titles, column_widths=[.7, .3])
    n_colors = len(colors)
    for i in range(n_rows):
        if i == 0:
            showlegend = True
        else:
            showlegend = False
        data_eigvecs = go.Scatter(
            y=eigvecs[:, -i - 1],
            x=np.arange(N),
            line=dict(
                color="lightslategrey",
            ),
            name="Eigenvectors",
            legendgroup="real-eigvec",
            showlegend=showlegend,
        )

        normalized_class_vec = labels[:, -i - 1] / np.sqrt(labels[:, -i-1].sum())
        data_class = go.Scatter(
            y=normalized_class_vec,
            x=np.arange(N),
            line=dict(color="firebrick"),
            name="Class vectors",
            legendgroup="class-vectors",
            showlegend=showlegend,
        )
        fig.add_trace(data_eigvecs, row=i + 1, col=1)
        fig.add_trace(data_class, row=i + 1, col=1)

        bar_trace = px.bar(
            y=["theoretical", "real"],
            x=[theoretical_alignments[i], real_alignments[i]],
            orientation="h",
            text_auto=True,
        )
        bar_trace.data[0].marker.color = ["firebrick", "lightslategrey"]
        fig.add_trace(bar_trace.data[0], row=i + 1, col=2)

    fig.update_layout(
        margin=dict(t=100, b=5, r=5, l=5),
        title_text=rf"Real and asymptotical eigenvectors alignments",
        width=1000,
        height=500,
        showlegend=True,
        font_size=14,
        title_font_size=20,
        font_family="Serif"
    )
    fig.update_annotations(font_size=20)
    fig.show()
    return fig

        


In [ ]:
real_alignments = []

for ind in range(n_outliers):
    current_eigvec = eigvec_outliers[:, -ind-1]
    class_vec = sorted_class_vectors[-ind-1]
    n_a = 1/class_vec.sum()
    alignment = (current_eigvec.T @ class_vec)**2 * n_a
    real_alignments.append(alignment)

real_alignments = real_alignments

theoretical_alignments = 1 - (sigma/ sorted_gamma[::-1])**2

In [ ]:
plot_eigvec_labels(save_path="homogene-eigvec.pdf", eigvecs=eigvec_outliers, labels=sorted_class_vectors.T, theoretical_alignments=theoretical_alignments, real_alignments=real_alignments)

# Hétérogène

Dans cette partie nous essayons quelques suggestions établies dans le rapport.

In [8]:
N = 1500
K = 3
# q_0 = .4
q_choices = [0.5, 0.1]
q = rng.choice(q_choices, N)
# q = q_0 * np.random.rand(N) + 0.1
proba = rng.randint(1, 5, K)
proba = proba / proba.sum()
M = 50 * np.diag(rng.rand(K))
B_scaled, class_vectors = generate_A_B(M, q, proba)


diag_sqrt_q = np.diag(1. / np.sqrt(q))


In [19]:
fig = project_and_3D_plot(B_scaled, class_vectors, "uniform", show=False)
fig.update_layout(title="Tracé problématique, clustering difficile.")
fig.show()
pio.write_image(fig, "hard-clustering.pdf")


In [21]:
B_renormalized = diag_sqrt_q @ B_scaled @ diag_sqrt_q

eigval, eigvec = np.linalg.eigh(B_renormalized)

fig = px.histogram(
        x=eigval, nbins=200, color_discrete_sequence=["firebrick"],
    )
fig.update_layout(
        margin=dict(t=100, b=5, r=5, l=5),
        title_text=rf"Distribution des valeurs propres de B renormalisée.",
        width=800,
        height=400,
        showlegend=False,
        font_family="Serif",
    )

fig.update_yaxes(title="")
fig.update_xaxes(title="")
fig.show()
pio.write_image(fig, "rescaled-eigval.pdf")

Ici nous avons effectivement retrouvé une distribution proche de celle d'une loi du demi-cercle. Visualisons alors les composantes dans l'espace des valeurs propres isolées.

### Répartition dans l'espace pour B renormalisée.

In [22]:
fig = project_and_3D_plot(B_renormalized, class_vectors, "uniform", show=False)
fig.update_layout(title="Répartition pour B renormalisée.")
pio.write_image(fig, "3d-rescaled.pdf")

Nous observons que pour $B$ renormalisée les clusters sont peut-être un peu plus facilement séparables que précédement.